In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to C:\Users\David
[nltk_data]     Isaac\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\David
[nltk_data]     Isaac\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [25]:
df = pd.read_csv("cleaner_data.csv")
print(df.size)
df.head()

88767


,Unnamed: 0,name,multiverse_id,color_identity,text,flavor,power
0,1,Ancestor's Chosen,NaN,['W'],First strike (This creature deals combat damag...,"""The will of all, by my hand done.""",4
1,2,Angel of Mercy,129465.0,['W'],Flying\r\nWhen Angel of Mercy enters the battl...,Every tear shed is a drop of immortality.,3
2,4,Angelic Blessing,129711.0,['W'],Target creature gets +3/+3 and gains flying un...,Only the warrior who can admit mortal weakness...,NaN
3,6,Angelic Chorus,129710.0,['W'],Whenever a creature enters the battlefield und...,The harmony of the glorious is a dirge to the ...,NaN
4,7,Angelic Wall,129671.0,['W'],Defender (This creature can't attack.)\r\nFlying,"""The Ancestor protects us in ways we can't beg...",0


In [26]:
import re

def clean_color(text):
     # as cores são branco(W), preto(B), azul(U), vermelho(R) ou verde(G)
    pattern = r"\['(W|B|U|R|G)'\]"
    
    match = re.search(pattern, text)
    
    if match:
        return match.group(1)
    return None

df["color"] = df["color_identity"].apply(clean_color)
df.drop("color_identity", axis=1, inplace = True)

In [27]:
df["color"].value_counts()

color
W    2632
R    2575
B    2563
G    2475
U    2436
Name: count, dtype: int64

In [28]:
df["counterspell"] = (df.text.str.contains(r"[Cc]ounter\s(?:it|target|all)") & # anulações
         (df.text.str.contains(r"[wW]ard(?:\s{|—])") == False)) # Ward é uma maneira de anulação/proteção mais transversal às cores e é ignorada


df["exile"] = (df.text.str.contains(r"[eE]xile\s(?:target|each|all|the|up\sto)") & # exilar algo do tablueiro
        (df.text.str.contains(r"the\stop") == False)) # Algumas cartas "compram" cartas exilando-as do topo do deck, isso nào conta como exilar algo do tabuleiro

df["fight"] = (df.text.str.contains(r"[Ff]ights")) # Criaturas lutarem é uma mecânica característica do verde


df["mill"] = (df.text.str.contains(r"[mM]ill")) # TODO: mill é uma palavra chave recente, antes era referida por descartar a carta do >TOPO DO BATALHO<, bom adicionar este texto também


df["scry"] = (df.text.str.contains(r"[sS]cry"))


df["tap"] = (df.text.str.contains(r"(?:\st|T)ap\s(?:it|target|each|all|or\suntap)")) # cartas que viram outras 


df["untap"] = (df.text.str.contains(r"[uU]ntap\s(?:it|target|each|all)")) # find untappers

"""
df.loc[df.text.str.contains("[dD]eathtouch") | # find creatures that have deathtouch
        df.text.str.contains("deals combat damage to a creature, destroy that creature", regex = False)] # or that have "derptouch"
"""

df["double_strike"] = (df.text.str.contains(r"[dD]ouble\sstrike"))


df["first_strike"] = (df.text.str.contains(r"[fF]irst\sstrike"))


df["flash"] = (df.text.str.contains(r"(?:f|\nF|^F)lash") & # some engineering to avoid incorrectly grabbing cards with Flash in the name
        (df.text.str.contains(r"[fF]lashback") == False)) # dont' want to capture flashback


df["flying"] = (df.text.str.contains(r"[fF]lying"))


df["haste"] = (df.text.str.contains(r"[hH]aste"))


df["hexproof"] = (df.text.str.contains(r"[hH]exproof"))


df["indestructible"] = (df.text.str.contains(r"[iI]ndestructible") &
                         (df.text.str.contains(r"loses\sindestructible") == False))


df["sacrifice"] = (df.text.str.contains(r"[sS]acrifice"))


df["lifelink"] = (df.text.str.contains(r"[lL]ifelink"))


df["menace"] = (df.text.str.contains(r"[mM]enace"))


df["protection"] = (df.text.str.contains(r"[pP]rotection\sfrom"))


df["prowess"] = (df.text.str.contains(r"[pP]rowess"))


df["reach"] = (df.text.str.contains(r"(?:\sr|\nR|^R)each") &
        (df.text.str.contains(r"can't be blocked except by creatures with flying or reach", regex = False) == False)) # don't want flying reminder text


df["trample"] = (df.text.str.contains(r"[tT]rample"))


df["vigilance"] = (df.text.str.contains(r"[vV]igilance"))


df["draw"] = (df.text.str.contains(r"(?:\sd|\nD|^D)raw"))


df["discard"] = (df.text.str.contains(r"[dD]iscard"))


df["damage"] = (df.text.str.contains(r"deals\s\d\sdamage"))


df["damage_prevention"] = (df.text.str.contains(r"[pP]revent\s"))


df["life_gain"] = (df.text.str.contains(r"gain(?:\s|s\s)\d+\slife"))


df["life_loss"] = (df.text.str.contains(r"loses") & 
                   df.text.str.contains(r"(?:their|\d+)\slife")) # capture both fixed and rational values


df["tokens"] = (df.text.str.contains(r"[cC]reate"))


df["destroy"] = (df.text.str.contains(r"[dD]estroy") &
                  (df.text.str.contains(r"don't\sdestroy\sit.") == False)) # reject indestructible's reminder text


df["return"] = (df.text.str.contains(r"[rR]eturn") &
        df.text.str.contains(r"owner's\s(?:hand|library)") & # capture hand or library bounce effects
        (df.text.str.contains(r"graveyard\sto") == False)) # exclude grave recursion


df["recursion"] = (df.text.str.contains(r"\sput|return") &
        df.text.str.contains(r"graveyard")&
        df.text.str.contains(r"hand|battlefield"))

In [29]:
df.head()

,Unnamed: 0,name,multiverse_id,text,flavor,power,color,counterspell,exile,fight,...,draw,discard,damage,damage_prevention,life_gain,life_loss,tokens,destroy,return,recursion
0,1,Ancestor's Chosen,NaN,First strike (This creature deals combat damag...,"""The will of all, by my hand done.""",4,W,False,False,False,...,False,False,False,False,True,False,False,False,False,False
1,2,Angel of Mercy,129465.0,Flying\r\nWhen Angel of Mercy enters the battl...,Every tear shed is a drop of immortality.,3,W,False,False,False,...,False,False,False,False,True,False,False,False,False,False
2,4,Angelic Blessing,129711.0,Target creature gets +3/+3 and gains flying un...,Only the warrior who can admit mortal weakness...,NaN,W,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,6,Angelic Chorus,129710.0,Whenever a creature enters the battlefield und...,The harmony of the glorious is a dirge to the ...,NaN,W,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,7,Angelic Wall,129671.0,Defender (This creature can't attack.)\r\nFlying,"""The Ancestor protects us in ways we can't beg...",0,W,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [30]:
# Exibir todas as colunas e amostras de valores únicos em texto
print("\nNomes das colunas e valores únicos:")
for coluna in df.columns:
    print(f"\nColuna: {coluna}")
    print(df[coluna].unique())


Nomes das colunas e valores únicos:

Coluna: Unnamed: 0
[    1     2     4 ... 76228 76231 76232]

Coluna: name
["Ancestor's Chosen" 'Angel of Mercy' 'Angelic Blessing' ...
 'Vastwood Fortification // Vastwood Thicket' 'Veteran Adventurer'
 'Vine Gecko']

Coluna: multiverse_id
[    nan 129465. 129711. ... 491866. 491869. 491870.]

Coluna: text
["First strike (This creature deals combat damage before creatures without first strike.)\r\nWhen Ancestor's Chosen enters the battlefield, you gain 1 life for each card in your graveyard."
 'Flying\r\nWhen Angel of Mercy enters the battlefield, you gain 3 life.'
 "Target creature gets +3/+3 and gains flying until end of turn. (It can't be blocked except by creatures with flying or reach.)"
 ... 'When Turntimber Ascetic enters the battlefield, you gain 3 life.'
 'Veteran Adventurer is also a Cleric, Rogue, Warrior, and Wizard.\r\nThis spell costs {1} less to cast for each creature in your party.\r\nVigilance'
 'The first kicked spell you cast ea

In [33]:
# Definir as stop words
stop_words = set(stopwords.words('english'))

# Função para limpar, tokenizar e remover stop words
def preprocess(text):
    text = re.sub(r'[^\w\s]', '', text.lower())  # Remove pontuação e converte para minúsculas
    tokens = word_tokenize(text)  # Tokeniza o texto
    filtered_tokens = [word for word in tokens if word not in stop_words]  # Remove as stop words
    return filtered_tokens

# Tokenizar a coluna 'flavor' e remover as stop words
tokenized_text = df['flavor'].apply(preprocess)

# Contar as frequências das palavras
word_freq_text = Counter(word for doc in tokenized_text for word in doc)

# Contar a coocorrência de pares de palavras
def count_coocurrences(corpus, window_size=1):
    coocurrence = Counter()
    for doc in corpus:
        for i, word in enumerate(doc):
            for j in range(i + 1, min(i + window_size + 1, len(doc))):
                coocurrence[(word, doc[j])] += 1
    return coocurrence

coocurrence_text = count_coocurrences(tokenized_text)

# Calcular o PMI para um par de palavras
def calculate_pmi(word1, word2, coocurrence, word_freq, corpus_size):
    coocurrence_count = coocurrence.get((word1, word2), 0)
    p_word1 = word_freq[word1] / corpus_size
    p_word2 = word_freq[word2] / corpus_size
    p_word1_word2 = coocurrence_count / corpus_size
    return np.log(p_word1_word2 / (p_word1 * p_word2)) if p_word1_word2 > 0 else 0

# Calculando o PMI para todos os pares de palavras
corpus_size_text = sum(word_freq_text.values())
pmi_results = {}

# Iterando por todos os pares únicos de palavras
for (word1, word2), count in coocurrence_text.items():
    pmi = calculate_pmi(word1, word2, coocurrence_text, word_freq_text, corpus_size_text)
    pmi_results[(word1, word2)] = pmi

# Exibir os resultados do PMI
for pair, pmi_value in pmi_results.items():
    print(f"PMI entre {pair[0]} e {pair[1]}: {pmi_value}")



PMI entre hand e done: 3.1354375035459854
PMI entre every e tear: 3.235978732767863
PMI entre tear e shed: 6.171086384505419
PMI entre shed e drop: 5.403831231791751
PMI entre drop e immortality: 5.9344594828539226
PMI entre warrior e admit: 5.261150644513641
PMI entre admit e mortal: 5.460001503258806
PMI entre mortal e weakness: 5.0181687509797674
PMI entre weakness e bolstered: 8.930191756407913
PMI entre bolstered e immortal: 8.624810106856732
PMI entre immortal e strength: 3.634377520077995
PMI entre harmony e glorious: 5.587834874768691
PMI entre glorious e dirge: 6.532296483609543
PMI entre dirge e wicked: 7.058389579506322
PMI entre ancestor e protects: 6.139903457068732
PMI entre protects e us: 4.142700013625867
PMI entre us e ways: 1.909107792118773
PMI entre ways e cant: 2.825398523992928
PMI entre cant e begin: 4.548165121734032
PMI entre begin e comprehend: 7.069439415692907
PMI entre comprehend e mystic: 6.5388111646307365
PMI entre mystic e elder: 6.62150288047585
PMI en

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [34]:
# Definindo palavras-chave para cores positivas, como exemplo
positive_colors = ['white', 'green', 'blue']  # Branco, Verde e Azul

# Função para identificar palavras associadas a cores positivas
def get_positive_words(pmi_results, positive_colors, top_n=10):
    positive_words = {}
    
    for (word1, word2), pmi in pmi_results.items():
        if any(color in word1 for color in positive_colors) or any(color in word2 for color in positive_colors):
            if word1 not in positive_words:
                positive_words[word1] = pmi
            else:
                positive_words[word1] += pmi
            if word2 not in positive_words:
                positive_words[word2] = pmi
            else:
                positive_words[word2] += pmi

    # Classificar as palavras por PMI e retornar as top_n palavras mais associadas
    sorted_positive_words = sorted(positive_words.items(), key=lambda x: x[1], reverse=True)
    return sorted_positive_words[:top_n]

# Exemplo de como obter as top 10 palavras mais positivas
top_positive_words = get_positive_words(pmi_results, positive_colors, top_n=20)
print("Top 20 palavras com maior positividade:")
for word, pmi in top_positive_words:
    print(f"{word}: {pmi}")


Top 20 palavras com maior positividade:
white: 146.93171505503025
green: 111.83233149099256
blue: 81.31316126351267
whitesun: 41.180835170011385
whitehot: 32.850164576136514
greenweaver: 28.367577018851136
greenwheel: 23.138498172046344
greenway: 20.499440842431085
greenseeker: 20.248126414150178
greenhouse: 19.078055161499925
told: 16.403906512073394
skyblue: 15.69366489815415
dim: 14.759188687993493
snowwhite: 14.67091548118344
whites: 12.600340368709247
wildest: 11.569249086023172
greenbelt: 11.569249086023172
turni: 11.569249086023172
groundskeeper: 11.569249086023172
wings: 11.28958630273801
